# GRT Final

This is the final development notebook of the GRT project.  In this notebook converge all the lessons learned during development.

In [26]:
import os
os.environ["OMP_NUM_THREADS"] = "1"

In [30]:
from gravray import *
from gravray.util import *
from gravray.stats import *
from gravray.spice import *
from gravray.plot import *
from gravray.sampling import *
from gravray.orbit import *

import skmonaco
import multiprocessing as mp

In [31]:
%matplotlib nbagg

## Initialize

In [32]:
dt=elTime(0)
Spice.loadKernels()
cneos=pd.read_csv("data/cneos_fireball_data_location.csv")
cneos.set_index("Name",inplace=True)
earth=Body("EARTH")
NP=mp.cpu_count()-1
print("Available number of processors: ",NP)

Available number of processors:  3


## Example conditions

In [6]:
#Site 
lon=59.8703**Angle.Deg #rad
lat=55.0958*Angle.Deg #rad
alt=23.3*Const.km #m

#Time
datestring="02/15/2013 03:20:34"
fmt="%m/%d/%Y %H:%M:%S"
t=Spice.str2tdb(datestring)

#Population
weights=[0.6]
locs=[
    [0.5,0.5,-2.0],
    [2.0,0.3,-2.6]
]
scales=[
    [1.3,0.7,0.5],
    [0.4,0.9,1.6]
]
angles=[
    [-40.0*Angle.Deg,-86.0*Angle.Deg,0.0*Angle.Deg],
    [+80.0*Angle.Deg,-109.0*Angle.Deg,0.0*Angle.Deg]
]
ranges=[
    [Util.fin2Inf(1e-2,360.0),Util.fin2Inf(360.0-1e-2,360.0)],
    [Util.fin2Inf(1e-2,360.0),Util.fin2Inf(360.0-1e-2,360.0)]
]
NEOs=MultiVariate([1,1,1,0,0])
NEOs.setUnflatten(weights,locs,scales,angles,ranges)

#Direction
Aloc=103.5*Angle.Deg
hloc=18.55*Angle.Deg
vimp=-18.6*Const.km/Const.s

Rdir=np.array([[Aloc],[hloc],[vimp]])

#Input conditions
print(f"Impact conditions:")
print(f"\tDate: {datestring}")
print(f"\tlon. {lon*Angle.Rad:.4g}, lat. {lat*Angle.Rad:.4g}, alt. {alt/Const.km:.4g} km")
print(f"\tAloc = {Aloc*Angle.Rad:.4g}, hloc = {lat*Angle.Rad:.4g}, vimp = {vimp/Const.km} km/s")

Impact conditions:
	Date: 02/15/2013 03:20:34
	lon. 61.54, lat. 55.1, alt. 23.3 km
	Aloc = 103.5, hloc = 55.1, vimp = -18.6 km/s


## Test routines

In [7]:
prob=GrtProb(t,earth,NEOs,verbose=True)
prob.setLocation(lon,lat,alt)
p=prob.calcPdir([Aloc,hloc,vimp])
p,prob.ph,prob.detJ,prob.Rdir

Impact conditions: np.array([[1.806415775814131],[0.32375857624494814],[-18600.0]])
Terminal elements: [  0.72231578   0.5297087    5.18079402 326.57568287 103.0150748
  24.56944643]
Unbound terminal elements: [ 0.89757661  0.11897496 -3.518794    2.27937803 -0.91414209 -2.61391129]


(5.228572267234881e-26,
 5.586619321805399e-05,
 -9.359098886202953e-22,
 [1.806415775814131, 0.32375857624494814, -18600.0])

In [8]:
prob=GrtProb(t,earth,NEOs,verbose=False)
prob.setLocation(lon,lat,alt)
p=prob.calcPsky(np.array([[Aloc],[hloc],[vimp]]))
p,prob.phs,prob.detJs,prob.selements,prob.Rsky

(array([5.22857227e-26]),
 array([5.58661932e-05]),
 array([-9.35909889e-22]),
 array([[  0.72231578,   0.5297087 ,   5.18079402, 326.57568287,
         103.0150748 ,  24.56944643]]),
 array([[ 1.80641578e+00,  3.23758576e-01, -1.86000000e+04]]))

In [9]:
prob=GrtProb(t,earth,NEOs,verbose=False)
p=prob.calcPimp(np.array([[lon],[lat],[alt],[Aloc],[hloc],[vimp]]))
p,prob.phs,prob.detJs,prob.selements,prob.Rimp

(array([5.22857227e-26]),
 array([5.58661932e-05]),
 array([-9.35909889e-22]),
 array([[  0.72231578,   0.5297087 ,   5.18079402, 326.57568287,
         103.0150748 ,  24.56944643]]),
 array([[ 1.07403440e+00,  9.61603114e-01,  2.33000000e+04,
          1.80641578e+00,  3.23758576e-01, -1.86000000e+04]]))

## Test integrations

### One dimensional integation (quad)

In [10]:
pint=MultiQuad(prob.calcPsky,["A","h","v"],integrator="quad")
i,di=pint.integrate({"A":[np.pi],"h":[np.pi/4],"v":np.array([-43.3,-11.1])*Const.km/Const.s})
print(f"Integral in one direction (integrator '{pint.nintegrator}'):",i,di,pint.fun_calls)

Integral in one direction (integrator 'quad'): 3.539251483977331e-23 3.2858412673551106e-23 21


### Two dimensional

In [11]:
pint=MultiQuad(prob.calcPsky,["A","h","v"],integrator="fixed_quad")
dt=elTime(0)
i,di=pint.integrate({"A":[0,2*np.pi],"h":[0,np.pi/2],"v":np.array([-18.6])*Const.km/Const.s})
dt=elTime(1)
print(f"Integral in two directions (integrator '{pint.nintegrator}'):",i,di,pint.fun_calls)

Elapsed time since last call: 133.912 ms
Integral in two directions (integrator 'fixed_quad'): 1.060624924834944e-25 None 5


### Three dimensional (approximate)

In [12]:
prob=GrtProb(t,earth,NEOs,verbose=False)
prob.setLocation(lon,lat,alt)
pint=MultiQuad(prob.calcPsky,["A","h","v"],integrator="fixed_quad",opt=dict(n=37))
dt=elTime(0)
i,di=pint.integrate({"A":[0,2*np.pi],"h":[0,np.pi/2],"v":np.array([-72.2,-11.1])*Const.km/Const.s})
dt=elTime(1)
print(i,di,pint.fun_calls)
print(f"Integral in three directions (integrator '{pint.nintegrator}'):",i,di,pint.fun_calls)

Elapsed time since last call: 592.452 ms
2.0715599473990289e-22 None 37
Integral in three directions (integrator 'fixed_quad'): 2.0715599473990289e-22 None 37


In [13]:
nint=MultiCube(prob.calcPsky,["A","h","v"])
dt=elTime(0)
i=nint.integrate({"A":[0,2*np.pi],"h":[0,np.pi/2],"v":np.array([-72.2,-11.1])*Const.km/Const.s})
dt=elTime(1)
print(f"Integral in three directions (integrator '{nint.nscheme}'):",i)

Elapsed time since last call: 504.775 ms
Integral in three directions (integrator 'dobrodeev_1978'): 1.6894810998515917e-21


### Three dimensional (exhaustive)

In [18]:
prob=GrtProb(t,earth,NEOs,verbose=False)
prob.setLocation(lon,lat,alt)
NCALLS=0
def fquad(*X):
    global NCALLS
    A,h,v=X
    p=prob.calcPdir([A,h,v])
    NCALLS+=1
    if (NCALLS%10000)==0:
        print(f"{NCALLS} calls... ")
        dt=elTime(1)
    return p

In [ ]:
print(datetime.now())
dt=elTime(0)
i=integrate.nquad(fquad,[[0,2*np.pi],[0,np.pi/2],np.array([-72.2,-11.1])*Const.km/Const.s],full_output=True)
dt=elTime(1)
print(datetime.now())
print(f"Integral in three directions (exhaustive):",i)

### Two dimensional (Monte Carlo)

In [20]:
NCALLS=0
def fmc2(X):
    global NCALLS
    A,h=X
    v=-18.6*Const.km/Const.s
    p=prob.calcPdir([A,h,v])
    NCALLS+=1
    if (NCALLS%1000)==0:
        print(f"{NCALLS} calls... ")
        dt=elTime(1)
    return p

In [36]:
NCALLS=0
N=int(1000/NP)*NP
print(f"Integrating with {N} points")
dt=elTime(0)
integ=skmonaco.mcquad(fmc2,npoints=N,xl=[0.0,0.0],xu=[2*np.pi,np.pi/2],nprocs=NP)
dt=elTime(1)
print("Two dimensional simple MC: ",integ)

1000 calls... 
Elapsed time since last call: 28.5263 s
2000 calls... 
Elapsed time since last call: 31.0011 s
3000 calls... 
Elapsed time since last call: 26.8007 s
Two dimensional simple MC:  (1.1734417941700956e-25, 1.496213492176886e-26)


In [38]:
NCALLS=0
N=int(1000/NP)*NP
print(f"Integrating with {N} points")
dt=elTime(0)
integ=skmonaco.mcmiser(fmc2,npoints=N,xl=[0.0,0.0],xu=[2*np.pi,np.pi/2],nprocs=NP)
dt=elTime(1)
print("Two dimensional MISER MC: ",integ)

Integrating with 999 points
Elapsed time since last call: 15.8231 s
Two dimensional MISER MC:  (9.630383939904805e-26, 1.1156840588569277e-26)


### Three dimensional (Monte Carlo)

In [24]:
NCALLSF=0
def fmc3(X):
    global NCALLSF
    A,h,v=X
    p=prob.calcPdir([A,h,v])
    NCALLSF+=1
    if (NCALLSF%100)==0:
        print(f"{NCALLSF} calls... ",)
        dt=elTime(1)
    return p

In [ ]:
NCALLSF=0
N=int(1000/NP)*NP
dt=elTime(0)
integ=skmonaco.mcquad(fmc3,npoints=N,
                      xl=[0.0,0.0,-72.2*Const.km/Const.s],
                      xu=[2*np.pi,np.pi/2,-11.1*Const.km/Const.s],
                      nprocs=NP)
dt=elTime(1)
print("Three dimensional simple MC: ",integ,NCALLSF)

In [ ]:
NCALLSF=0
N=int(1000/NP)*NP
dt=elTime(0)
integ=skmonaco.mcmiser(fmc3,npoints=N,
                      xl=[0.0,0.0,-72.2*Const.km/Const.s],
                      xu=[2*np.pi,np.pi/2,-11.1*Const.km/Const.s],
                      nprocs=NP)
dt=elTime(1)
print("Three dimensional MISER MC: ",integ,NCALLSF)

--End--

In [1]:
"""
pint=MultiQuad(prob.calcPsky,["A","h","v"],integrator="quad")
dt=elTime(0)
i,di=pint.integrate({"A":[0,2*np.pi],"h":[0,np.pi/2],"v":np.array([-18.6])*Const.km/Const.s})
dt=elTime(1)
print(f"Integral in three directions (integrator '{pint.nintegrator}'):",i,di,pint.fun_calls)
pint=MultiQuad(prob.calcPsky,["A","h","v"],integrator="quad")
dt=elTime(0)
i,di=pint.integrate({"A":[0,2*np.pi],"h":[0,np.pi/2],"v":np.array([-18.6])*Const.km/Const.s})
dt=elTime(1)
print(i,di,pint.fun_calls)
pint=MultiQuad(prob.calcPsky,["A","h","v"],integrator="quad")
dt=elTime(0)
i,di=pint.integrate({"A":[0,2*np.pi],"h":[0,np.pi/2],"v":np.array([-72.2,-11.1])*Const.km/Const.s})
dt=elTime(1)
print(i,di,pint.fun_calls)
"""
1

1